## 第9章 最適化アラカルト

### 9.2 ソルバーの威力

In [ ]:
import numpy as np
from pulp import LpProblem,LpMaximize,LpVariable,LpBinary,lpDot
def make(n):
    np.random.seed(1)  # 乱数シードを設定することで、同じ乱数を生成
    w = 1 + np.random.rand(n)
    p = w + np.random.randn(n) * 0.1
    m = LpProblem(sense=LpMaximize)
    v = [LpVariable('x%05d'%i,cat=LpBinary) for i in range(n)]
    m += lpDot(p, v)
    m += lpDot(w, v) <= int(n*1.25)
    return m

In [ ]:
m = make(100)
%timeit -r3 -n3 m.solve()

### 9.3 ナップサック問題の結果の図示

In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
from ortoolpy import knapsack  # 厳密解法
def greedy(siz, prf, capa):  # 貪欲法
    p, r = 0, []
    for i in range(len(siz)-1, -1, -1):
        if siz[i] <= capa:
            capa -= siz[i]
            p += prf[i]
            r.append(i)
    return p, r
def stingy(siz, prf, capa):  # 吝嗇法
    p, r = 0, []
    rm = siz.sum()
    for i in range(len(siz)):
        if 0 < rm-siz[i] <= capa and siz[i] <= capa:
            capa -= siz[i]
            p += prf[i]
            r.append(i)
        rm -= siz[i]
    return p, r
np.random.seed(0)
n = 100  # アイテム数
siz = np.random.uniform(0.1, 1.0, n)  # 大きさ
prf = siz * np.random.lognormal(1, 0.1, n)  # 価値
eff = prf / siz  # 効率
siz, prf, eff = np.array([siz,prf,eff]).T[eff.argsort()].T
pl1, pl2, pl3, rl1, rl2, rl3 = [], [], [], [], [], []
for capa in np.arange(2, siz.sum() + 2, 2):
    p1, r1 = knapsack(siz, prf, capa)
    pl1.append(p1)
    rl1.append([int(i in r1) for i in range(n)])
    p2, r2 = greedy(siz, prf, capa)
    pl2.append(p2)
    rl2.append([int(i in r2) for i in range(n)])
    p3, r3 = stingy(siz, prf, capa)
    pl3.append(p3)
    rl3.append([int(i in r3) for i in range(n)])
plt.imshow(1 - np.array(rl1).T, cmap='gray')
plt.show()
plt.imshow(1 - np.array(rl2).T, cmap='gray')
plt.show()
plt.imshow(1 - np.array(rl3).T, cmap='gray');

In [ ]:
plt.ylim((0.9, 1.02))
plt.hlines(1, 0, len(pl1))
plt.plot(np.array(pl2[2:]) / np.array(pl1[2:]));

In [ ]:
plt.ylim((0.9, 1.02))
plt.hlines(1, 0, len(pl1))
plt.plot(np.array(pl3[2:]) / pl1[2:]);

### 9.4 ミニサムとミニマックスとは

In [ ]:
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pulp import LpProblem, LpBinary, lpDot, lpSum, value
from ortoolpy import addvar, addvars
商品数, ユーザ数 = 1000, 100
np.random.seed(1)
df = pd.DataFrame(np.random.rand(商品数, ユーザ数),
    index=[f'商品{i:03}' for i in range(商品数)],
    columns=[f'ユーザ{j:02}' for j in range(ユーザ数)])
addvars(df)
df[:2]

In [ ]:
it = [100, 200, 500, 1000]  # 商品数リスト
tm = []
for n in it:
    dfs = df[:n]
    m1 = LpProblem()  # ミニサム問題
    m1 += lpDot(dfs.T[:-1].sum(), dfs.Var)  # 合計(サム)
    m1 += lpSum(dfs.Var) <= n // 2
    m1.solve()
    m2 = LpProblem()  # ミニマックス問題
    y = addvar()
    # y >= max(ユーザj の価値)
    for j in range(ユーザ数):
        m2 += y >= lpDot(dfs.iloc[:, j], dfs.Var)
    m2 += y  # 合計(マックス)
    m2 += lpSum(dfs.Var) <= n // 2
    m2.solve()
    tm.append((m1.solutionTime, m2.solutionTime))
plt.plot(it, tm)
plt.legend(['min-sum','min-max'], loc='upper left');

### 9.6 ビンパッキング問題に対するアプローチの比較

In [ ]:
import numpy as np
from pulp import LpProblem, lpSum, lpDot
from ortoolpy import addvar, addvars, addbinvars
# n1, n2 = 3, 20 # 箱数, アイテム数
n1, n2 = 3, 12 # 箱数, アイテム数
np.random.seed(1)
sz = np.random.randint(1, 1000000, n2)  # サイズ

In [ ]:
# アプローチ0
m = LpProblem()
x = np.array(addbinvars(n1, n2))
y = addvars(n1)
z = addvar()
m += z  # 目的関数
for j in range(n2):
    m += lpSum(x[:, j]) == 1
for i in range(n1):
    m += y[i] == lpSum(sz * x[i])
    m += y[i] - sum(sz)/n1 <= z
%time m.solve()

In [ ]:
# アプローチ1
m = LpProblem()
x = np.array(addbinvars(n1, n2))
y = addvars(n1)
for j in range(n2):
    m += lpSum(x[:, j]) == 1
for i in range(n1):
    m += y[i] == lpSum(sz * x[i])
#    m += y[i] <= 2646137
    m += y[i] <= 1482001
%time m.solve()

In [ ]:
# アプローチ2
m = LpProblem()
x = np.array(addbinvars(n1, n2))
y = addvars(n1)
m += lpSum(y)  # 目的関数（なくてもよい）
for j in range(n2):
    m += lpSum(x[:, j]) == 1
for i in range(n1):
    m += y[i] == lpSum(sz * x[i])
#    m += y[i] <= 2646137
    m += y[i] <= 1482001
    if i:
        m += y[i-1] <= y[i]
%time m.solve()

In [ ]:
# アプローチ3
m = LpProblem()
x = np.array(addbinvars(n1, n2))
y = addvars(n1)
z = addvar()  # max
m += z  # 目的関数
for j in range(n2):
    m += lpSum(x[:, j]) == 1
for i in range(n1):
    m += y[i] == lpSum(sz * x[i])
    m += y[i] <= z
%time m.solve()

In [ ]:
# アプローチ4
m = LpProblem(sense=LpMaximize)
x = np.array(addbinvars(n1, n2))
y = addvars(n1)
z = addvar()  # min
m += z  # 目的関数
for j in range(n2):
    m += lpSum(x[:, j]) == 1
for i in range(n1):
    m += y[i] == lpSum(sz * x[i])
    m += y[i] >= z
%time m.solve()

In [ ]:
# アプローチ5
m = LpProblem()
x = np.array(addbinvars(n1, n2))
y = addvars(n1)  # sum
z = addvars(n1)  # diff
w = addvars(n1)  # cost
m += lpSum(w)  # 目的関数
for j in range(n2):
    m += lpSum(x[:, j]) == 1
for i in range(n1):
    m += y[i] == lpSum(sz * x[i])
    m += z[i] >=  (y[i]-sum(sz)/n1)
    m += z[i] >= -(y[i]-sum(sz)/n1)
    m += w[i] >= 0.2 * z[i]
    m += w[i] >= 0.5 * z[i] - 7.5
    m += w[i] >=       z[i] - 25
%time m.solve()

### 9.7 線形緩和問題とは

In [ ]:
from pulp import (LpProblem, LpMaximize, LpBinary,
                  LpVariable, lpDot, value)
m = LpProblem(sense=LpMaximize)  # 数理モデル
x,y,z = [LpVariable(c, cat=LpBinary) for c in 'xyz'] # 変数
m += lpDot([7,8,9], [x,y,z])  # 目的関数
m += lpDot([6,7,8], [x,y,z]) <= 14  # 制約条件
m.solve()  # 求解
print([value(v) for v in [x,y,z]])  # 出力

In [ ]:
from pulp import LpProblem,LpMaximize,LpVariable,lpDot,value
m = LpProblem(sense=LpMaximize)  # 数理モデル
# 変数
x,y,z = [LpVariable(c,lowBound=0,upBound=1) for c in 'xyz']
m += lpDot([7,8,9], [x,y,z])  # 目的関数
m += lpDot([6,7,8], [x,y,z]) <= 14  # 制約条件
m.solve()  # 求解
print([value(v) for v in [x,y,z]])  # 出力

In [ ]:
from pulp import LpContinuous
for v in m.variables():
    v.cat = LpContinuous

### 9.8 緩和固定法

In [ ]:
import pandas as pd
start = pd.datetime(2020, 1, 1)
end = pd.datetime(2020, 1, 14)
window = pd.Timedelta('7D')
fix = pd.Timedelta('3D')
last = end - window + fix - pd.Timedelta('1D')
rng = pd.date_range(start, last, freq=fix)
for i, d in enumerate(rng):
    t = d.date()
    print(f'{i+1}回目 ~{t}:fix ~{t+window}:window')

### 9.11 双対問題

In [ ]:
import dual

In [ ]:
%%dual
min c^T x
A x >= b
x >= 0

In [ ]:
%%dual
max b^T y
A^T y <= c
y >= 0

In [ ]:
%%dual
min c^T x
A x = b
x >= 0

In [ ]:
%%dual
min c^T x
A x >= b

In [ ]:
%%dual
max c^T x + d^T z
A x - P z >= b
Q z <= f
x >= 0

In [ ]:
%%dual
min - b^T y + f^T w
-A^T y >= c
P^T y + Q^T w = d
y >= 0
w >= 0

### 9.12 モンテカルロ法を用いた最短路の計算

In [ ]:
%matplotlib inline
import numpy as np, networkx as nx, matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 5, 1.8
m = 4  # 横の頂点数
g = nx.Graph()
for i in range(m):
    if i==0:
        g.add_edge(i, i+m, prob=[1], time=[1.9])  # 0-> 4
    else:
        g.add_edge(i, i+m, prob=[0.8,0.2], time=[1,6]) # 縦
    if i < m-1:
        g.add_edge(i, i+1, prob=[1], time=[2])  # 横
        g.add_edge(i+m, i+m+1, prob=[1], time=[2])  # 横
n = g.number_of_nodes()
pos = {i:[i%m, i//m] for i in range(n)}
nx.draw_networkx(g, pos, node_color='w')

In [ ]:
def monte_min(g, s, t, nn=1000):
    n = g.number_of_nodes()
    dd = [np.inf] * n  # 到達時間
    bb = [False] * n  # 探索済み
    for _, d in g.edges.items():
        d['log'] = (np.random.multinomial(1, d['prob'], nn)
                    * d['time']).sum(axis=1)
    nx = t  # 次の点
    dd[nx] = 0
    bb[nx] = True
    while not bb[s] and not np.isposinf(dd[nx]):
        for nd in g.adj[nx]:
            dd[nd] = min(dd[nd], np.mean([calcmin(dd,
                g.adj[nd], i) for i in range(nn)]))
        nx = np.argmin([np.inf if bb[i] else dd[i]
                        for i in range(n)])
        bb[nx] = True
    return np.round(dd, 2)
def calcmin(dd, dc, i):
    return min([dd[nd]+d['log'][i] for nd,d in dc.items()])
np.random.seed(1)
monte_min(g, 0, 7)

### 9.13 パズルを最適化で解く
[パズルを最適化で解く](opt4puzzle/index.ipynb)